In [6]:
#load dependencies
import pandas as pd
import numpy as np
import os
import re
import tensorflow as tf
import sklearn
import pickle
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [15]:
#Load model and variables for encoding in model
label_encoder = LabelEncoder()
model = load_model("LSTM_actor_quotes_model.h5")
with open('max_quote_length.pickle', 'rb') as handle:
        max_quote_length_dic = pickle.load(handle)
max_quote_length=max_quote_length_dic['max_quote_length']
label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)
print(max_quote_length_dic)

{'max_quote_length': 43}


In [16]:
#Function for pre-processing text to take out common words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Preprocessing steps
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Tokenization
    words = word_tokenize(text)
    # Removing punctuation and converting to lowercase
    words = [word.lower() for word in words if word.isalnum()]
    # Removing stopwords
    words = [word for word in words if word not in stop_words]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Skynet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Skynet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Skynet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
#Function for predicting actor from quote using the model
def ActorPredictor(phrase):
    phrase_ls=[]
    for i in phrase:
        phrase_ls.append(preprocess_text(i))
    tokenizer = Tokenizer(num_words=10000)  # Adjust based on your vocabulary size
    #tokenizer.fit_on_texts(df_top_actors['Quote'])  # texts should be your input data
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    sequences = tokenizer.texts_to_sequences(phrase_ls)  # Convert text to integer sequences
    phrase_count = pad_sequences(sequences, maxlen=max_quote_length)
    actors_pred_all = model.predict(phrase_count)
    actors_pred=[]
    for i in actors_pred_all:
        i=list(i)
        actors_pred.append(i.index(max(i)))
    return label_encoder.inverse_transform(actors_pred)


In [19]:
#Sample implementation of the program
ActorPredictor(["terminator"])


1/1 [==============================] - 0s 10ms/step


array(['Arnold Schwarzenegger'], dtype=object)